In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
cats = [
    '00:00-01:00', '01:00-02:00', '02:00-03:00', '03:00-04:00', '04:00-05:00','05:00-06:00', '06:00-07:00', '07:00-08:00',
    '08:00-09:00', '09:00-10:00', '10:00-11:00','11:00-12:00', '12:00-13:00', '13:00-14:00','14:00-15:00', '15:00-16:00', 
    '16:00-17:00', '17:00-18:00', '18:00-19:00', '19:00-20:00','20:00-21:00', '21:00-22:00', '22:00-23:00','23:00-24:00'
    ]

In [3]:
df = pd.read_csv('data/sample_data_hour.csv').drop(columns=['Unnamed: 0'])

## Classify sessions

In [4]:
df_tmp = df[df['duration_min'] < 30]
quantiles = df_tmp['duration_min'].quantile([.33, .66, .99, 1])

In [5]:
#labeling of groups
df_tmp['group'] = 'high'
df_tmp['group'][df_tmp['duration_min'] <= quantiles[.99]] = 'high'
df_tmp['group'][df_tmp['duration_min'] <= quantiles[.66]] = 'mid'
df_tmp['group'][df_tmp['duration_min'] <= quantiles[.33]] = 'low'      
df_tmp.head(10)

user_id        date  year month        day  hour  minute  second  \
0     8953  2016-12-24  2016   Dec   Saturday     6      11      31   
1     3633  2017-02-08  2017   Feb  Wednesday    16      22      53   
2     3633  2016-12-09  2016   Dec     Friday    17      36      52   
3     3633  2018-06-03  2018   Jun     Sunday    21      20      16   
4     3633  2016-09-16  2016   Sep     Friday     9      16      39   
5     3633  2018-08-16  2018   Aug   Thursday    22      25      49   
6     3633  2016-08-06  2016   Aug   Saturday    13       0      21   
7     3633  2017-04-13  2017   Apr   Thursday    14       6      17   
8     3633  2018-04-01  2018   Apr     Sunday    14      35      26   
9     3633  2017-11-04  2017   Nov   Saturday    14      39      36   

   hour_period  duration_min group  
0  06:00-07:00         1.500  high  
1  16:00-17:00         0.333   low  
2  17:00-18:00         0.167   low  
3  21:00-22:00         0.167   low  
4  09:00-10:00         0.167   low  
5  22:00-23:00         1.167   mid  
6  13:00-14:00         0.667   mid  
7  14:00-15:00         1.667  high  
8  14:00-15:00         1.167   mid  
9  14:00-15:00         0.167   low

## Filter 'high' sessions

In [6]:
high = df_tmp[df_tmp.group == 'high']

In [7]:
high = high.groupby(['hour_period', 'day'])['duration_min'].mean().reset_index(name='Avg duration')

In [8]:
all_hours = pd.DataFrame(cats, columns=['hour_period'])

all_hours_daily_df = \
  pd.merge(high[['day']].drop_duplicates(),
           all_hours[['hour_period']].drop_duplicates(),
           how = 'cross')

high = \
  high.\
  merge(all_hours_daily_df,
  how='right').\
  fillna({'Avg duration': 0}).\
  reset_index(drop = True)

high

hour_period        day  Avg duration
0    00:00-01:00     Friday      3.676600
1    01:00-02:00     Friday      2.867000
2    02:00-03:00     Friday      0.000000
3    03:00-04:00     Friday      7.388833
4    04:00-05:00     Friday      5.265600
..           ...        ...           ...
163  19:00-20:00  Wednesday      3.003108
164  20:00-21:00  Wednesday      4.943944
165  21:00-22:00  Wednesday      3.292133
166  22:00-23:00  Wednesday      5.269687
167  23:00-24:00  Wednesday      3.338833

[168 rows x 3 columns]

In [9]:
high.to_csv('data/high_group.csv')

## Plot

In [134]:
df_389 = df[(df.user_id == 389)].sort_values(['date', 'hour_period']).reset_index(drop=True)
df_389

user_id        date  year month       day  hour  minute  second  \
0         389  2017-06-03  2017   Jun  Saturday    23       6      58   
1         389  2017-06-04  2017   Jun    Sunday    12       7      18   
2         389  2017-06-04  2017   Jun    Sunday    13      45      51   
3         389  2017-06-04  2017   Jun    Sunday    14       9      11   
4         389  2017-06-04  2017   Jun    Sunday    15      53      30   
...       ...         ...   ...   ...       ...   ...     ...     ...   
1970      389  2018-12-21  2018   Dec    Friday    14      52      25   
1971      389  2018-12-21  2018   Dec    Friday    15      14       1   
1972      389  2018-12-21  2018   Dec    Friday    17       1      16   
1973      389  2018-12-21  2018   Dec    Friday    20      24      13   
1974      389  2018-12-21  2018   Dec    Friday    20      47      55   

      hour_period  duration_min  
0     23:00-24:00         0.333  
1     12:00-13:00         0.167  
2     13:00-14:00         0.167  
3     14:00-15:00         0.667  
4     15:00-16:00         0.550  
...           ...           ...  
1970  14:00-15:00         0.167  
1971  15:00-16:00         0.500  
1972  17:00-18:00         0.500  
1973  20:00-21:00         3.683  
1974  20:00-21:00         0.333  

[1975 rows x 10 columns]

In [135]:
df_rigdeplot = df_389[['day', 'hour_period','duration_min']]

In [136]:
all_hours = pd.DataFrame(cats, columns=['hour_period'])

all_hours_daily_df = \
  pd.merge(df_rigdeplot[['day']].drop_duplicates(),
           all_hours[['hour_period']].drop_duplicates(),
           how = 'cross')

df_rigdeplot = \
  df_rigdeplot.\
  merge(all_hours_daily_df,
  how='right').\
  fillna({'duration_min': 0}).\
  reset_index(drop = True)

In [137]:
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

df_rigdeplot = df_rigdeplot.groupby(['day', 'hour_period'])['duration_min'].mean().reset_index(name='Avg duration')
df_rigdeplot['day'] = pd.Categorical(df_rigdeplot['day'], categories=days, ordered=True)
df_rigdeplot.sort_values(['day', 'hour_period'], inplace=True)

In [138]:
df_rigdeplot

day  hour_period  Avg duration
24  Monday  00:00-01:00      1.229250
25  Monday  01:00-02:00      0.000000
26  Monday  02:00-03:00      0.000000
27  Monday  03:00-04:00      0.176800
28  Monday  04:00-05:00      0.814625
..     ...          ...           ...
91  Sunday  19:00-20:00      0.911250
92  Sunday  20:00-21:00      1.702267
93  Sunday  21:00-22:00      1.263765
94  Sunday  22:00-23:00      2.753500
95  Sunday  23:00-24:00      2.956800

[168 rows x 3 columns]

In [171]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=7, cols=1, shared_xaxes=True, shared_yaxes=True)
xpoints = {}

for i, day in enumerate(df_rigdeplot.day.unique()):
    periods = []
    df = df_rigdeplot[(df_rigdeplot.day == day)]
    fig.add_trace(go.Scatter(x=df['hour_period'].astype(dtype=str), 
                            y=df['Avg duration'], fill='tozeroy',  name=day),
            row=i+1, col=1)

    for period in df.hour_period.unique():

        extreme_dur = high[(high.day == days[i]) & (high.hour_period == period)]['Avg duration'].values[0]
        
        duration = df[df.hour_period ==  period]['Avg duration'].values[0]

        if (extreme_dur > 0) & (duration > 0):

            if (duration >= extreme_dur):
                periods.append(period)

    xpoints[i] = periods


shapes_lst = []

for i, vals in xpoints.items():
    if len(vals) > 0:
        for val in vals:
            if i == 0:
                shapes_lst.append(dict(type='rect', xref='x', yref='y',
                        x0=val, x1=val, y0=0.4, y1=5, line=dict(
                                    color="Red",
                                    width=20,
                                ), opacity=0.4, line_width=1, layer='below'))    
            else:
                shapes_lst.append(dict(type='rect', xref=f'x{i+1}', yref=f'y{i+1}',
                            x0=val, x1=val, y0=0.4, y1=5, line=dict(
                                    color="Red",
                                    width=10,
                                ), opacity=0.4, line_width=20, layer='below'))

fig.update_layout(
        shapes=shapes_lst, height=1000, title_text="Stacked Subplots"
)

fig.for_each_yaxis(lambda x: x.update(showgrid=False))
fig.show()


Wednesday
Thursday
Thursday
{0: [], 1: [], 2: ['00:00-01:00'], 3: ['05:00-06:00', '23:00-24:00'], 4: [], 5: [], 6: []}
